In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib

In [19]:
# Use non-interactive backend for file saving
matplotlib.use('Agg')

# --- Simulation Parameters ---
NUM_PARTICLES = 30
BOX_SIZE = 500.0
BOX_HEIGHT = 200.0 
DT = 0.1

# --- Global Interaction Parameters ---
LJ_STRENGTH = 5000.0
ALIGNMENT_RADIUS = 50.0
ALIGNMENT_STRENGTH = 0.05
MOBILITY = 1.0
MAX_V_FORCE = 5.0
COS_45 = np.cos(np.pi / 4.0)

# --- NEW: Weathervane Effect Parameter ---
# Controls how strongly they align against the flow.
WEATHERVANE_STRENGTH = 0.1 

# --- Flow Parameters ---
MAX_FLOW_SPEED = 2.5 

In [20]:
class Particle:
    """Represents a single active particle."""
    def __init__(self, x, y, angle, is_ghost=False):
        self.pos = np.array([x, y], dtype=float)
        self.angle = angle
        self.dir = np.array([np.cos(self.angle), np.sin(self.angle)])
        self.is_ghost = is_ghost
        self.active = True 
        
        # Properties 
        self.period = 10.0
        self.pusher_duration = 9.0
        self.pusher_strength = 500.0
        self.puller_strength = 50.0
        self.propulsion_speed = 0.5
        self.color = 'C0' 

        self.force = np.zeros(2)
        self.neighbor_dirs = []
        self.fixed_time = None 

    def update_dir(self):
        self.dir[0] = np.cos(self.angle)
        self.dir[1] = np.sin(self.angle)

    def reset(self):
        self.force.fill(0.0)
        self.neighbor_dirs = []

class Simulation:
    def __init__(self, flow_type):
        self.flow_type = flow_type
        self.particles = []
        self.time = 0.0
        self.upstream_exits = 0
        self.downstream_exits = 0
        
        # Initialize particles near the right
        for _ in range(NUM_PARTICLES):
            x = np.random.rand() * BOX_SIZE * 0.5 + BOX_SIZE * 0.4
            y = np.random.rand() * BOX_HEIGHT * 0.8 + BOX_HEIGHT * 0.1
            # Random initial angle
            angle = np.random.rand() * 2 * np.pi
            self.particles.append(Particle(x, y, angle))

    def get_flow_velocity(self, y):
        y_clamped = np.clip(y, 0, BOX_HEIGHT)
        h = BOX_HEIGHT
        if self.flow_type == 'poiseuille':
            v_x = 4 * MAX_FLOW_SPEED * (y_clamped/h) * (1.0 - y_clamped/h)
        elif self.flow_type == 'shielded':
            base = 4 * (y_clamped/h) * (1.0 - y_clamped/h)
            term1 = y_clamped * (h - y_clamped)
            term2 = (y_clamped - h/2.0)**2
            v_x = (MAX_FLOW_SPEED * 16.0 / (h**4)) * term1 * (term2 + 500.0) 
        return np.array([v_x, 0.0])

    def get_interaction_force(self, p1, p2, r_vec, r_mag_sq):
        r_mag = np.sqrt(r_mag_sq)
        r_hat = r_vec / r_mag
        
        if r_mag_sq < 20.0**2:
             lj_mag = LJ_STRENGTH / (r_mag_sq**3.0 + 1e-6)
             f_lj = -lj_mag * r_hat 
        else:
             f_lj = np.zeros(2)

        t_eff = p2.fixed_time if p2.is_ghost else self.time
        p2_time = t_eff % p2.period
        p2_is_pusher = p2_time < p2.pusher_duration

        dot_2 = np.dot(p2.dir, -r_hat)
        in_cone_2 = np.abs(dot_2) > COS_45
        base_sign = 1.0 if in_cone_2 else -1.0
        mag = (base_sign * p2.pusher_strength) if p2_is_pusher else \
              (-base_sign * p2.puller_strength)
        f_aniso = (mag / (r_mag_sq + 1.0)) * (-r_hat)

        return f_lj + f_aniso

    def step(self):
        # 1. Create Ghosts
        ghosts = []
        for p in self.particles:
            if not p.active: continue
            if p.pos[1] < 100.0: 
                g = Particle(p.pos[0], -p.pos[1], 0, is_ghost=True)
                g.dir = np.array([p.dir[0], -p.dir[1]])
                g.fixed_time = (self.time + np.random.rand()*p.period)
                ghosts.append(g)
            if p.pos[1] > BOX_HEIGHT - 100.0:
                g = Particle(p.pos[0], 2*BOX_HEIGHT - p.pos[1], 0, is_ghost=True)
                g.dir = np.array([p.dir[0], -p.dir[1]])
                g.fixed_time = (self.time + np.random.rand()*p.period)
                ghosts.append(g)

        # 2. Interactions
        for p in self.particles: p.reset()

        for i in range(NUM_PARTICLES):
            p1 = self.particles[i]
            if not p1.active: continue
            for j in range(i + 1, NUM_PARTICLES):
                p2 = self.particles[j]
                if not p2.active: continue
                r_vec = p2.pos - p1.pos
                r_mag_sq = np.sum(r_vec**2)
                if 1e-2 < r_mag_sq < ALIGNMENT_RADIUS**2:
                     f1 = self.get_interaction_force(p1, p2, r_vec, r_mag_sq)
                     f2 = self.get_interaction_force(p2, p1, -r_vec, r_mag_sq)
                     p1.force += f1
                     p2.force += f2
                     p1.neighbor_dirs.append(p2.dir)
                     p2.neighbor_dirs.append(p1.dir)

        for p in self.particles:
            if not p.active: continue
            for g in ghosts:
                r_vec = g.pos - p.pos
                r_mag_sq = np.sum(r_vec**2)
                if 1e-2 < r_mag_sq < (ALIGNMENT_RADIUS * 1.5)**2:
                    p.force += self.get_interaction_force(p, g, r_vec, r_mag_sq)
                    p.neighbor_dirs.append(g.dir)

        # 3. Update Particles
        for p in self.particles:
            if not p.active: continue

            # --- NEW: Total Torque Calculation ---
            torque_total = 0.0
            
            # A. Neighbor Alignment Torque
            if p.neighbor_dirs:
                avg_dir = np.sum(p.neighbor_dirs, axis=0)
                target_angle = np.arctan2(avg_dir[1], avg_dir[0])
                diff = (target_angle - p.angle + np.pi) % (2*np.pi) - np.pi
                torque_total += diff * ALIGNMENT_STRENGTH

            # B. Weathervane Torque (Align against flow)
            # Torque is proportional to local flow speed and sin(angle to upstream)
            # Upstream is angle PI. sin(pi - theta) = sin(theta)
            v_flow_mag = self.get_flow_velocity(p.pos[1])[0]
            torque_wv = WEATHERVANE_STRENGTH * v_flow_mag * np.sin(p.angle)
            torque_total += torque_wv

            # Apply torque + noise
            p.angle += torque_total * DT + (np.random.rand()-0.5)*0.2
            p.update_dir()

            # --- Velocity Update ---
            v_propel = p.dir * p.propulsion_speed
            v_flow = self.get_flow_velocity(p.pos[1])
            v_force = p.force * MOBILITY
            v_force_mag = np.linalg.norm(v_force)
            if v_force_mag > MAX_V_FORCE:
                v_force = (v_force / v_force_mag) * MAX_V_FORCE
                
            p.pos += (v_propel + v_flow + v_force) * DT

            # Boundaries & Exits
            if p.pos[1] < 1.0: p.pos[1] = 1.0
            if p.pos[1] > BOX_HEIGHT - 1.0: p.pos[1] = BOX_HEIGHT - 1.0
            
            if p.pos[0] < 0:
                p.active = False
                self.upstream_exits += 1
                p.pos = np.array([-1000.0, -1000.0]) 
            elif p.pos[0] > BOX_SIZE:
                p.active = False
                self.downstream_exits += 1
                p.pos = np.array([-1000.0, -1000.0])

        self.time += DT

# --- Animation & Running ---
def run_simulation(flow_type, filename, title):
    print(f"Starting {title}...")
    sim = Simulation(flow_type)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), width_ratios=[4, 1])
    
    ax1.set_xlim(0, BOX_SIZE)
    ax1.set_ylim(-20, BOX_HEIGHT+20)
    ax1.set_aspect('equal')
    ax1.set_title(title)
    ax1.axhline(0, color='k', linewidth=2)
    ax1.axhline(BOX_HEIGHT, color='k', linewidth=2)
    ax1.set_xlabel("Channel Length (X)")
    ax1.set_ylabel("Channel Height (Y)")
    
    pos = np.array([p.pos for p in sim.particles])
    dirs = np.array([p.dir for p in sim.particles]) * 10.0
    quiver = ax1.quiver(pos[:,0], pos[:,1], dirs[:,0], dirs[:,1], color='C0', scale=120)
    
    upstream_text = ax1.text(0.02, 1.15, 'Upstream Exits: 0', 
                             transform=ax1.transAxes, color='green', fontsize=12, fontweight='bold')
    downstream_text = ax1.text(0.65, 1.15, 'Downstream Exits: 0', 
                               transform=ax1.transAxes, color='red', fontsize=12, fontweight='bold')

    y_vals = np.linspace(0, BOX_HEIGHT, 100)
    v_vals = np.array([sim.get_flow_velocity(y)[0] for y in y_vals])
    ax2.plot(v_vals, y_vals, 'k--')
    ax2.fill_betweenx(y_vals, 0, v_vals, alpha=0.2, color='gray')
    ax2.set_ylim(0, BOX_HEIGHT)
    ax2.set_xlim(0, MAX_FLOW_SPEED * 1.1)
    ax2.set_ylabel("Channel Height (Y)")
    ax2.set_xlabel("Flow Velocity (X)")
    ax2.set_title("Flow Profile")
    ax2.grid(True, alpha=0.3)

    def update(frame):
        for _ in range(3): sim.step()
        pos = np.array([p.pos for p in sim.particles])
        dirs = np.array([p.dir for p in sim.particles]) * 10.0
        quiver.set_offsets(pos)
        quiver.set_UVC(dirs[:,0], dirs[:,1])
        upstream_text.set_text(f'Upstream Exits: {sim.upstream_exits}')
        downstream_text.set_text(f'Downstream Exits: {sim.downstream_exits}')
        return quiver, upstream_text, downstream_text

    anim = FuncAnimation(fig, update, frames=1500, interval=20, blit=True)
    try:
        anim.save(filename, writer='ffmpeg', fps=210, dpi=120)
        print(f"Saved {filename}")
    except Exception as e:
        print(f"Error saving {filename}: {e}")
    plt.close(fig)

In [21]:
if __name__ == "__main__":
    run_simulation('poiseuille', 'flow_poiseuille.mp4', 'Simulation 1: Poiseuille Flow')
    run_simulation('shielded', 'flow_shielded.mp4', 'Simulation 2: Shielded (M-Profile) Flow')

Starting Simulation 1: Poiseuille Flow...
Saved flow_poiseuille.mp4
Starting Simulation 2: Shielded (M-Profile) Flow...
Saved flow_shielded.mp4
